In [1]:
from Pyfhel import Pyfhel, PyCtxt, PyPtxt


In [2]:
HE = Pyfhel()
HE.contextGen(p=2**16 +1 , m=2**12)
HE.keyGen()

In [3]:
import base64

def ctxt2utf(ctxt):
    ctxt_bytes = PyCtxt.to_bytes(ctxt)
    ctxt_bytes_base64 = base64.b64encode(ctxt_bytes) 
    ctxt_string = ctxt_bytes_base64.decode('utf-8')
    return ctxt_string

ca = HE.encryptFrac(5.0)
cb = HE.encryptFrac(3.0)

ca_string = ctxt2utf(ca)
cb_string = ctxt2utf(cb)


def utf2ctxt(ctxt_string_restored):
    print(type(ctxt_string_restored))
    ctxt_restored = PyCtxt()
    ctxt_bytes_base64_restored = ctxt_string_restored.encode('utf-8')
    print(type(ctxt_bytes_base64_restored))
    ctxt_bytes_restored = base64.b64decode(ctxt_bytes_base64_restored)
    print(type(ctxt_bytes_restored))
    ctxt_restored.from_bytes(ctxt_bytes_restored, float)
    print(type(ctxt_restored))
    return ctxt_restored

    

#str(ca_string)print(str(ca_string))


In [4]:
from influxdb import InfluxDBClient
import base64
json_body_ca = [
    {
        "measurement": "testMeas",
        "tags": {
            "device": "001",
            "location": "Messina"
        },
        "fields": {
            "value": ca_string
        }
    }
]

json_body_cb = [
    {
        "measurement": "testMeas",
        "tags": {
            "device": "002",
            "location": "Messina"
        },
        "fields": {
            "value": cb_string
        }
    }
]


client = InfluxDBClient(host='influxdb', port=8086, username='admin', password='temp', database='security_db', ssl=False)
client.write_points(json_body_ca)
client.write_points(json_body_cb)

result = client.query('select value from testMeas;')
#print("Result: {0}".format(result))




In [9]:
results = client.query('SELECT value FROM testMeas')
#print(results.raw)
results.get_points()
points = results.get_points()


[ca_restored, cb_restored] = [utf2ctxt(point['value']) for point in points]
print(HE.decryptFrac(ca_restored))
print(HE.decryptFrac(cb_restored))
cresult = HE.add(ca_restored, cb_restored)
result = HE.decryptFrac(cresult)
assert(result == 8)


# for point in points:
    
#     print(type(point['value']))
#     ca_string_restored = point['value']
#     assert(ca_string_restored == ca_string)
#     print('Worked !')

#     ca_bytes_base64_restored = ca_string_restored.encode('utf-8')
#     assert(ca_bytes_base64_restored == ca_bytes_base64)
#     print('Worked !')

#     ca_bytes_restored = base64.b64decode(ca_bytes_base64_restored)
#     assert(ca_bytes_restored == ca_bytes) 
#     print('Worked !')

#     ca_restored.from_bytes(ca_bytes_restored, float)
#     print(HE.decryptFrac(ca_restored))

<class 'str'>
<class 'bytes'>
<class 'bytes'>
<class 'Pyfhel.PyCtxt.PyCtxt'>
<class 'str'>
<class 'bytes'>
<class 'bytes'>
<class 'Pyfhel.PyCtxt.PyCtxt'>
5.0
3.0


In [ ]:
#client.drop_measurement('testMeas')